# READ ME 

## Necessary Installs

In [ ]:
#!pip install html5lib
#!pip install python-youtube    

##  Necessary API KEY

In [ ]:
# Ce notebook nécessite une clé API gratuite pour l'api Youtube V3
# Il suffit de la demander sur ce lien : https://console.cloud.google.com/apis/api/youtube.googleapis.com/
# La doc est ici : https://developers.google.com/youtube/v3

# Imports

In [ ]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
from configparser import ConfigParser
from datetime import datetime
from pyyoutube import Api
import locale
import requests
import sys
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import pickle
import isodate
import re
import os

If you're new to scraping data, a good guide can be found here : https://towardsdatascience.com/all-pandas-read-html-you-should-know-for-scraping-data-from-html-tables-a3cbb5ce8274

# Function definition

In [ ]:
def parse_html_document(soup_loc):
    #Parse the HTML document
    full_dic = {}

    for i in soup_loc.find_all(
            class_=
            'content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1'):
        i.find_all(name='a')

        date_ = i.contents[-1]

        #loop to find the 2 links in a given bracket
        for j in i.find_all(name='a'):
            #Parfois il n'y a pas de channel name... donc je met ces deux variables défaut et je teste dans le if
            channel_url = None
            channel_name = None
            #tester si on a les infos pour les channels
            if "channel" in j.get("href"):
                #lien "channel"
                channel_url = j.get("href")
                #condition pour virer une erreur bizarre de "list index out of range"
                if len(i.find_all(name='a')) == 2:
                    #Add channel name
                    channel_name = i.find_all(name='a')[1].text

            if "watch" in j.get("href"):
                #lien vid_url
                vid_url = j.get("href")

            #update the dic
            full_dic[date_] = {
                "channel": channel_url,
                "vid_url": vid_url,
                "channel_name": channel_name,
                "vid_title": i.find_all(name='a')[0].text
            }

    return full_dic

In [ ]:
# Function to convert French timestamp to useful timestamp
def convert_french_timestamp(timestamp):
    # Set the French locale
    locale.setlocale(locale.LC_TIME, "fr_FR.UTF-8")
    
    # Define the timestamp format
    format_str = "%d %m %Y à %H:%M:%S %Z"
    
    # Dictionary to map French month abbreviations to English
    month_translation = {
        "janv.": "01",
        "févr.": "02",
        "mars": "03",
        "avr.": "04",
        "mai": "05",
        "juin": "06",
        "juil.": "07",
        "août": "08",
        "sept.": "09",
        "oct.": "10",
        "nov.": "11",
        "déc.": "12",
    }
    
    # Replace the French month abbreviation with the English abbreviation
    for french, english in month_translation.items():
        timestamp = timestamp.replace(french, english)
    
    # Parse the timestamp string into a datetime object
    dt = datetime.strptime(timestamp, format_str)
    
    return dt

In [ ]:
def clean_dico(dico):
    # Mainteant on clean le dico
    df = pd.DataFrame.from_dict(dico, orient='index')
    df_index_reset = df.reset_index().rename(columns={"index": "date"})

    df_index_reset.date = df_index_reset.date.astype(str)
    
    # Apply the function to the DataFrame column and convert to datetime
    df_index_reset['date'] = df_index_reset['date'].apply(convert_french_timestamp)
    df_index_reset['date'] = pd.to_datetime(df_index_reset['date'])
    
    #Let's now set our date column as index
    df_w_dates = df_index_reset.set_index("date")

    #Create a column with the video ID extracted from vid_url. Allows interaction with the Youtube API v3
    df_w_dates["vid_id"] = df_w_dates.vid_url.str.extract(
        r"((?<=(v|V)/)|(?<=be/)|(?<=(\?|\&)v=)|(?<=embed/))([\w-]+)")[3]
    df_cleaned = df_w_dates.copy()

    return df_cleaned

In [ ]:
def fetch_details(list_of_50_ids):

    #Query Google API
    tmp_query_results = api.get_video_by_id(
        video_id=list_of_50_ids, parts='snippet,contentDetails,statistics')
    response_dic = tmp_query_results.to_dict()

    #nombre de résultats recus via l'api
    vid_details_dic = {}

    #total_results sometimes differs from the list of 50 ids, i don't know why yet
    for i in range(0, tmp_query_results.pageInfo.totalResults):

        #get video id to join later on initial dataframe
        vid_id_ = response_dic["items"][i]["id"]

        #duration ISO 8601 format, utiliser parse_duration de la librarie from isodate import parse_duration
        duration_raw = response_dic["items"][i]["contentDetails"]["duration"]
        duration = isodate.parse_duration(duration_raw)

        #Vid views_infos
        nbviews = response_dic["items"][i]["statistics"]["viewCount"]
        nblikes = response_dic["items"][i]["statistics"]["likeCount"]
        nbcomments = response_dic["items"][i]["statistics"]["commentCount"]

        #snippet data
        date_of_vid_publication = response_dic["items"][i]["snippet"][
            "publishedAt"]
        vid_description = response_dic["items"][i]["snippet"]["description"]
        tags = response_dic["items"][i]["snippet"]["tags"]

        #appends chosen results into a dictionnary, for which the primary key is the video_id,
        # to later be able to join on our primary df
        vid_details_dic[vid_id_] = {
            "duration": duration,
            "date_of_vid_publication": date_of_vid_publication,
            "tags": tags,
            "nbviews": nbviews,
            "nblikes": nblikes,
            "nbcomments": nbcomments,
            "vid_description": vid_description
        }
    return vid_details_dic

In [ ]:
def create_details(df_initial_clean):
    fifty_ids_batch = []
    bigbrain = pd.DataFrame()

    # Rappel, len(df_initial_clean.vid_id) pour moi == 48000...
    for i in range(0, len(df_initial_clean.vid_id)):
        fifty_ids_batch.append(df_initial_clean.vid_id[i])

        #tous les 50 vid_id, ie 0->49, stocker les id_s, faire appel à l'API, stocker le résultat
        #condition i>45 parce que apparemment 1%49==0

        if i % 49 == 0 and i > 45:

            #Ici, on a 50 ids dans la liste, donc on peut utiliser notre fct get_vid_details dessus
            df_tmp_results = pd.DataFrame.from_dict(
                fetch_details(fifty_ids_batch), orient='index')

            #récupérer le résultat et concatener dans notre bigbrain dataframe
            bigbrain = pd.concat([bigbrain, df_tmp_results], axis=0)

            #reset de la liste
            fifty_ids_batch = []

    return bigbrain

In [ ]:
def initialize_api(key):
    api = Api(api_key=key)
    return (api, api.get_authorization_url())

# Extract YT history data from HTML file

In [ ]:
#Set local environment variables 
os.environ["YT_HISTORY_PATH"] = "/Users/WDescamps/Desktop/code_projects/not_pushed_yet/YouTube_Analysis/watch-history.html"


In [ ]:
#Get local environment variables
yt_hist_file_path = os.environ["YT_HISTORY_PATH"]
print(yt_hist_file_path)

In [ ]:
with open(yt_hist_file_path, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

In [ ]:
# On fait appel a nos fonctions

In [ ]:
dictionnary = parse_html_document(soup)

In [ ]:
dictionnary

In [ ]:
df_propre = clean_dico(dictionnary)
df_propre.reset_index(inplace=True)

In [ ]:
df_propre.head(5)

## Sauvegarder en local c'est cool

In [ ]:
csv_path = 'YT_data.csv'

df_propre.to_csv(csv_path)

# Get  Youtube API data to enrich our data

## Récupérer la sauvegarde c'est cool aussi

In [ ]:
# Récupérer le df en mémoire, on sait jamais...
#df_propre=pd.read_csv(full_csv_path, index_col="date")

## Hop on continue

In [ ]:
# Je récupère ma clé API en local 
# Cf guide sur ce lien https://towardsdatascience.com/keeping-credentials-safe-in-jupyter-notebooks-fbd215a8e311

parser = ConfigParser()
_ = parser.read('youtube_api.cfg')
yt_api_key= parser.get('my_api', 'auth_key')

In [ ]:
yt_api_key

In [ ]:
api, auth_url= initialize_api(yt_api_key)

## Get video details

In [ ]:
# UNCOMMENT THESE CELL TO RUN THEM - Batch calls Youtube API, 50 video ids at a time,
# and creates a new df containing YT API data
# This can take quite a while 

df_details_raw=create_details(df_propre)


In [ ]:
df_details_raw.head(5)

## Sauvegarde en csv 

In [ ]:
df_details = df_details_raw.copy()

In [ ]:
df_details["vid_id"]=df_details.index

In [ ]:
df_details.date_of_vid_publication=pd.to_datetime(df_details.date_of_vid_publication)
df_details=df_details.sort_values("date_of_vid_publication", ascending=False)

full_csv_path = 'YT_data_detailed.csv'
df_details.to_csv(full_csv_path)

In [ ]:
df_details

## Clean video details

### Si sauvegarde existe, Réimport du df_propre initial pour l'enrichir ensuite avec df_detailed

In [ ]:
df_propre_path = 'YT_data.csv'
df_propre=pd.read_csv(df_propre_path)
#Virer la colonne ajoutée dans le read_csv
df_propre.pop(df_propre.columns[0])
df_propre.head(1)

## Lets now join our dataframes !

In [ ]:
df_details = pd.read_csv(
    'YT_data_detailed.csv',
    index_col="vid_id")
df_details.drop("Unnamed: 0", axis=1) # Degeu, je sais, mais bon...

In [ ]:
print(df_propre.shape, df_details.shape)

In [ ]:
# Join with original dataframe
df_joined = df_propre.merge(df_details,
                            how="left",
                            left_on="vid_id",
                            right_index=True)
df_joined.drop_duplicates("vid_id", inplace=True)
df_joined.date = pd.to_datetime(df_joined.date)

In [ ]:
print(df_joined.columns, "\n", df_joined.shape, "\n")

In [ ]:
df_joined.head(5)

## Write to csv

In [ ]:
full_csv_path = 'YT_data_joinde.csv'
df_joined.to_csv(full_csv_path)

In [ ]:
# How many rows without tag? Probably before the creation of tags by the platform
df_joined.tags.isna().sum()

## Split each year into a dic

In [ ]:
# Creates Yearly Dataframes and puts them in a dictionnary with "{year}" as a key
list_of_years = df_joined.date.dt.year.unique()
dic_years = {}

for year_ in list_of_years:
    dic_years[str(year_)] = df_joined[df_joined.date.dt.year ==
                                      year_].reset_index(drop=True)

# Now let's have fun analysing our data

In [ ]:
#How many unique videos have i watched per channel? Top 10 of all time
df_joined.channel_name.value_counts().head(20)

In [ ]:
def get_most_watched_tags_in_year(year):
    
    import re
    #Créer une nouvelle colonne avec un dictionnaire pour chaque row avec la cellule ci-dessous
    #Si possible, ajouter 1 où une clé existe deja, sinon crée autre clé
    result = {}
    for tag in dic_years[year]["tags"]:
        list_of_tags= list(re.findall(r"'\s*([^']+?)\s*'", str(tag)))
        for item in list_of_tags:

            if f"{item}" not in result:
                result[str(item)]=1
            else:
                result[str(item)]+=1
                
    tag_results_df=pd.DataFrame.from_dict(result, orient="index", columns=["count"]).reset_index()
    sorted_results=tag_results_df.sort_values("count", ascending=False)
    return sorted_results

In [ ]:
#Maintenant on peut jouer a voir chaque tag par année. On pourrait voir comment ils progressent chaque année
get_most_watched_tags_in_year("2020").head(10)

## Yearly watch history

In [ ]:
dic_years["2022"].head(1)

In [ ]:
#Plot yearly consumption

In [ ]:
annee="2017"

nb_vids_per_week = dic_years[annee].groupby(
    dic_years[annee].date.dt.to_period("W"))[["vid_id"]].agg('count').reset_index()

In [ ]:
nb_vids_per_week.plot()

#### How did my time spent on the platform evolve YoY?

In [ ]:
#Plot YoY consuption on the same graph

#### Which tags are the most frequent? How do they evolve

#### What about descriptions?

#### What about time spent?

#### Do i watch new or old videos?

## Add Captions (In progress)

In [ ]:
### Try to get caption/subtitles a single video NOT WORKING

Caption = api.get_captions_by_video(
    video_id='Z_QnyfEokg8',
    caption_id='AUieDaaAJvMo8nUKiqUuP6X0QbXKb9xcpjTY4mT5-Ny41JpLys8')

In [ ]:
#https://stackoverflow.com/questions/14061195/how-to-get-transcript-in-youtube-api-v3